**[Microchallenges Home Page](https://www.kaggle.com/learn/microchallenges)**

---


# Intro

Data scientists tend to focus on **prediction** because that's where conventional machine learning excels. But real world decision-making involves both prediction and **optimization**.  After predicting what will happen, you decide what to do about it.

Optimization gets less attention than it deserves. So this micro-challenge will test your optimization skills as you write a function to improve how airlines set prices.

![Imgur](https://i.imgur.com/AKrbLMR.jpg)


# The Problem

You recently started Aviato.com, a startup that helps airlines set ticket prices. 

Aviato's success will depend on a function called `pricing_function`.  This notebook already includes a very simple version of `pricing_function`.  You will modify `pricing_function` to maximize the total revenue collected for all flights in our simulated environment.

For each flight, `pricing_function` will be run once per (simulated) day to set that day's ticket price. The seats you don't sell today will be available to sell tomorrow, unless the flight leaves that day.

Your `pricing_function` is run for one flight at a time, and it takes following inputs:
- **Number of days until the flight**
- **Number of seats they have left to sell**
- **A variable called `demand_level` that determines how many tickets you can sell at any given price. **

The quantity you sell at any price is:
> quantity_sold = demand_level - price

Ticket quantities are capped at the number of seats available.

Your function will output the ticket price.

You learn the `demand_level` for each day at the time you need to make predictions for that day. For all days in the future, you only know `demand_level` will be drawn from the uniform distribution between 100 and 200.  So, for any day in the future, it is equally likely to be each value between 100 and 200.

In case this is still unclear, some relevant implementation code is shown below.

# The Simulator
We will run your pricing function in a simulator to test how well it performs on a range of flight situations.  **Run the following code cell to set up your simulation environment:**

In [1]:
import sys
sys.path.append('../input')
from flight_revenue_simulator import simulate_revenue, score_me


In case you want to check your understanding of the simulator logic, here is a simplified version of some of the key logic (leaving out the code that prints your progress). If you feel you understand the description above, you can skip reading this code.

```
def _tickets_sold(p, demand_level, max_qty):
        quantity_demanded = floor(max(0, p - demand_level))
        return min(quantity_demanded, max_qty)

def simulate_revenue(days_left, tickets_left, pricing_function, rev_to_date=0, demand_level_min=100, demand_level_max=200):
    if (days_left == 0) or (tickets_left == 0):
        return rev_to_date
    else:
        demand_level = uniform(demand_level_min, demand_level_max)
        p = pricing_function(days_left, tickets_left, demand_level)
        q = _tickets_sold(demand_level, p, tickets_left)
        return _total_revenue(days_left = days_left-1, 
                              tickets_left = tickets_left-q, 
                              pricing_function = pricing_function, 
                              rev_to_date = rev_to_date + p * q,
                              demand_level_min = demand_level_min,
                              demand_level_max = demand_level_max
                             )
```

# Your Code

Here is starter code for the pricing function.  If you use this function, you will sell 10 tickets each day (until you run out of tickets).

In [2]:
#Initial function -> Average revenue across all flights is $6073
def pricing_function(days_left, tickets_left, demand_level):
    """Sample pricing function"""
    price = demand_level - 10
    return price

#1st trial -> Average revenue across all flights is $6678
def pricing_function(days_left, tickets_left, demand_level):
    """Sample pricing function"""
    if days_left>10:
        return 190
    price = demand_level - tickets_left/days_left
    return price

#2nd trial -> Average revenue across all flights is $7199
def pricing_function(days_left, tickets_left, demand_level):
    """Sample pricing function"""
    if days_left>10:
        return 183
    price = demand_level - tickets_left/days_left
    return price

#3rd trial: a more sofisticated function -> Average revenue across all flights is $7483
import numpy as np

#Creating two matrix Q & V that will store optimum number of tickets to sell for all situations
n_demand_levels = 11
min_demand_level = 100
max_demand_level = 200
demand_levels = np.linspace(min_demand_level, max_demand_level, n_demand_levels)

max_tickets = 200
max_days = 150

# Q indices are: n_sold in day, tickets_left to start day, demand_level, days_left
Q = np.zeros([max_tickets, max_tickets, n_demand_levels, max_days])
# V indices are: n_left and n_days
V = np.zeros([max_tickets, max_days])

#Let's fill the matrix Q and V
for days_left in range(1, max_days):
    for tickets_left in range(max_tickets):
        for tickets_sold in range(tickets_left):
            for demand_index, demand_level in enumerate(demand_levels):
                #Setting the price and not negative
                price = max(demand_level - tickets_sold, 0)
                current_revenu = price * tickets_sold
                Q[tickets_sold, tickets_left, demand_index, days_left] = current_revenu + V[tickets_left-tickets_sold, days_left-1]
        expected_total_rev_from_best_quantity_at_each_demand_level = Q[:, tickets_left, :, days_left].max(axis=0)
        V[tickets_left, days_left] = expected_total_rev_from_best_quantity_at_each_demand_level.mean()         
        
#It's time to define the function using our matrixes Q and V
def pricing_function(days_left, tickets_left, demand_level):
    if days_left == 1:
        return max(demand_level - tickets_left, demand_level/2)
    demand_level_index = np.abs(demand_level - demand_levels).argmin()
    day_index = days_left - 1 # arrays are 0 indexed
    tickets_index = int(tickets_left)  # in case it comes in as float, but need to index with it
    relevant_Q_vals = Q[:, tickets_index, demand_level_index, day_index]
    desired_quantity = relevant_Q_vals.argmax()# offset 0 indexing
    price = demand_level - desired_quantity
    return price

To see a small example of how your code works, test it with the following function:

In [3]:
simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function, verbose=True)

7 days before flight: Started with 50 seats. Demand level: 137. Price set to $135. Sold 2 tickets. Daily revenue is 271. Total revenue-to-date is 271. 48 seats remaining
6 days before flight: Started with 48 seats. Demand level: 148. Price set to $140. Sold 8 tickets. Daily revenue is 1122. Total revenue-to-date is 1393. 40 seats remaining
5 days before flight: Started with 40 seats. Demand level: 159. Price set to $147. Sold 12 tickets. Daily revenue is 1765. Total revenue-to-date is 3158. 28 seats remaining
4 days before flight: Started with 28 seats. Demand level: 174. Price set to $160. Sold 14 tickets. Daily revenue is 2241. Total revenue-to-date is 5399. 14 seats remaining
3 days before flight: Started with 14 seats. Demand level: 114. Price set to $114. Sold 0 tickets. Daily revenue is 0. Total revenue-to-date is 5399. 14 seats remaining
2 days before flight: Started with 14 seats. Demand level: 142. Price set to $129. Sold 13 tickets. Daily revenue is 1681. Total revenue-to-dat

7243.146715065566

You can try simulations for a variety of values.

Once you feel good about your pricing function, run it with the following cell to to see how it performs on a wider range of flights.

In [4]:
score_me(pricing_function)

Ran 200 flights starting 100 days before flight with 100 tickets. Average revenue: $18406
Ran 200 flights starting 14 days before flight with 50 tickets. Average revenue: $8451
Ran 200 flights starting 2 days before flight with 20 tickets. Average revenue: $2643
Ran 200 flights starting 1 days before flight with 3 tickets. Average revenue: $431


<IPython.core.display.Javascript object>

Average revenue across all flights is $7483


---
**[Microchallenges Home Page](https://www.kaggle.com/learn/microchallenges)**


*Want to discuss your solution or hear what others have done?  Visit the [Learn Discussion forum](https://www.kaggle.com/general/62469) to chat with other Learners.*